# Computing and Visualizing Access and Equity

In this workbook we will combine the travel times we calculated in the previous notebook with the demographics visualized in the first to get an understanding of how Calgary's transit access to two opportunities (hospitals and childcare) are distributed.

Our basic workflow will be this:
1. Using the travel time matrices and destination data, calculate a metric for each DA zone
2. Compute a population weighted sum for different population groups to see how these benefits are distributed on average.

## Import and Read Demographics

In [ ]:
import pandas as pd
import geopandas as gpd
import altair as alt
demographic_columns = ["pop_total", "vismin_vismin", "lico_lico", "fam_onemother"]
demographic_names = {
    "pop_total": "Everyone",
    "vismin_vismin": "Visible Minority",
    "lico_lico": "Low Income",
    "fam_onemother": "Single Mother Households"
}
demographics = pd.read_csv("data/demographics.csv", dtype={"dauid":str})
demographics.head()

## Access to the Nearest Hospital
Next, lets load in our travel time matrix for the AM peak with access to hospitals. To perform a weighted summary over different demographic groups we need to do the following:

1. Decide what exactly we should do with the `NaN` values. Let's for now fill them in with a 1-hour extra value.
2. Since we are looking for *minimum travel time*, we simply group by each origin and take the minimum value.
3. Join in the demographics data so that we have everything nicely together.
4. Calculate the weighted average travel time to hopsitals for different demographic groups

In [ ]:
hosp_am = pd.read_csv("data/mx_hospitals_am.csv", dtype={"from_id":str})
# Step 1
hosp_am["travel_time"] = hosp_am["travel_time"].fillna(180)
# Step 2
hosp_am = hosp_am[["from_id", "travel_time"]].groupby("from_id", as_index=False).min()
# Step 3
hosp_am = pd.merge(hosp_am, demographics, left_on="from_id", right_on="dauid")
# Step 4
# Let's keep only the totals columns and the travel time that we need
hosp_am_avg = hosp_am[["travel_time", "pop_total", "vismin_vismin", "lico_lico", "fam_onemother"]].copy()
# Now we normalize the demographic columns so we can do our weighting properly
for c in demographic_columns:
    hosp_am_avg[c] = hosp_am_avg[c]/hosp_am_avg[c].sum()
# Finally we multiply our travel time by these fractional amounts and sum to get a weighted average
hosp_am_avg = hosp_am_avg[demographic_columns].multiply(hosp_am_avg["travel_time"], axis="index").sum().to_frame().reset_index()
# Rename our columns to be something prettier
hosp_am_avg.columns = ["demographic", "avg_travel_time"]
# Finally we do some pretty names for our plots
hosp_am_avg["demo_name"] = hosp_am_avg["demographic"].map(demographic_names)
hosp_am_avg

Now we have our weighted sums, lets make a plot to show them. A bar chart covers most of what we want here, so lets use a slightly fancier version: A Lollipop chart.

In [ ]:
sticks = alt.Chart(hosp_am_avg).mark_bar(color="lightgrey", height=4).encode(
    alt.X("avg_travel_time:Q", title="Average Travel Time (min)"),
    alt.Y("demo_name:N", title="", sort=["Everyone"])
)

lollipop = alt.Chart(hosp_am_avg).mark_circle(color="#823BA0", size=250, opacity=1).encode(
    alt.X("avg_travel_time:Q", title="Average Travel Time (min)"),
    alt.Y("demo_name:N", title="", sort=["Everyone"])
)

(sticks+lollipop).properties(
    title="Average Travel Time to Hospitals (Mornings)",
    width=400,
    height=100
).configure(
    font="Ubuntu"
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure_axisY(
    labelFontWeight="bold"
)

## Comparing Two Travel Times
We can also compare two travel time situations. For example, how big is the disparity between AM peak and evening service, and who is affected by this disparity the most?

Let's build another set of access measures for the evening and compare

In [ ]:
hosp_pm = pd.read_csv("data/mx_hospitals_pm.csv", dtype={"from_id":str})
hosp_pm["travel_time"] = hosp_pm["travel_time"].fillna(180)
hosp_pm = hosp_pm[["from_id", "travel_time"]].groupby("from_id", as_index=False).min()
hosp_pm = pd.merge(hosp_pm, demographics, left_on="from_id", right_on="dauid")
# Let's keep only the totals columns and the travel time that we need
hosp_pm_avg = hosp_pm[["travel_time", "pop_total", "vismin_vismin", "lico_lico", "fam_onemother"]].copy()
# Now we normalize the demographic columns so we can do our weighting properly
for c in demographic_columns:
    hosp_pm_avg[c] = hosp_pm_avg[c]/hosp_pm_avg[c].sum()
# Finally we multiply our travel time by these fractional amounts and sum to get a weighted average
hosp_pm_avg = hosp_pm_avg[demographic_columns].multiply(hosp_pm_avg["travel_time"], axis="index").sum().to_frame().reset_index()
# Rename our columns to be something prettier
hosp_pm_avg.columns = ["demographic", "avg_travel_time"]
# Finally we do some pretty names for our plots
hosp_pm_avg["demo_name"] = hosp_pm_avg["demographic"].map(demographic_names)
hosp_pm_avg

In [ ]:
sticks = alt.Chart(hosp_pm_avg).mark_bar(color="lightgrey", height=4).encode(
    alt.X("avg_travel_time:Q", title="Average Travel Time (min)"),
    alt.Y("demo_name:N", title="", sort=["Everyone"])
)

lollipop = alt.Chart(hosp_pm_avg).mark_circle(color="#823BA0", size=250, opacity=1).encode(
    alt.X("avg_travel_time:Q", title="Average Travel Time (min)"),
    alt.Y("demo_name:N", title="", sort=["Everyone"])
)

(sticks+lollipop).properties(
    title="Average Travel Time to Hospitals (Evenings)",
    width=400,
    height=100
).configure(
    font="Ubuntu"
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False
).configure_axisY(
    labelFontWeight="bold"
)

Now we can take a difference between the two

In [ ]:
hosp_am_pm = pd.merge(
    hosp_am_avg[["demographic", "avg_travel_time"]], 
    hosp_pm_avg[["demographic", "avg_travel_time"]], 
    on="demographic", 
    suffixes=["_am", "_pm"]
)
hosp_am_pm["delta"] = hosp_am_pm["avg_travel_time_pm"] - hosp_am_pm["avg_travel_time_am"]
hosp_am_pm["demo_name"] = hosp_am_pm["demographic"].map(demographic_names)
hosp_am_pm

And then we can plot this difference much as we did above

In [ ]:
sticks = alt.Chart(hosp_am_pm).mark_bar(color="lightgrey", height=4).encode(
    alt.X("delta:Q", title="Travel Time Increase (min)"),
    alt.Y("demo_name:N", title="", sort=["Everyone"])
)

lollipop = alt.Chart(hosp_am_pm).mark_circle(color="#559613", size=250, opacity=1).encode(
    alt.X("delta:Q", title="Travel Time Increase (min)"),
    alt.Y("demo_name:N", title="", sort=["Everyone"])
)

(sticks+lollipop).properties(
    title="Additional Evening Travel Time to Hospitals",
    width=400,
    height=100
).configure(
    font="Ubuntu",
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=14
).configure_axisY(
    labelFontWeight="bold"
).configure_title(
    fontSize=16,
    anchor="start"
)